In [4]:
%pip install openAI
%pip install langchain
%pip install typing-inspect==0.8.0 typing_extensions==4.5.0
%pip install pydantic==1.10.8

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import openai
from langchain import *
from pydantic import *
import os 
import pandas as pd 
with open("secret.txt") as f:
    key=f.read()
os.environ['OPENAI_API_KEY']=key[:-1] #for some reason there's a new line, so we remove it



# Let's try LangChain

Literally just grabbed the example from the website and modified. still have no idea how this works but whatever (for now) 


## Putting everything together

In [6]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List



# Define your desired data structure.
class extracted_data(BaseModel):
    descriptions: list = Field(description="The descriptions for the shape. This should be a list of one or more strings.")
    
    





input =pd.read_csv("data/pre/rotate_6.csv")

parser = PydanticOutputParser(pydantic_object=extracted_data)

info_query_generic = "Here's a partial transcript of someone describing an image to someone else. Extract verbatim the parts of the text that describe of the image. \
Do not include fillers like 'it looks like' or 'sorta'  \
  or conversational words that aren't descriptive. \n\n \
  As an example, if the transcript was 'It looks like a magician, and, uhh, I think he's got a rabbit.', the response would be ['a magician','he's got a rabbit'].\n\n \
     As an example, if the transcript was 'yes, sorry, big triangle is on the right, like an arm', the response would be ['big triangle is on the right, like an arm'].\n\n \
      Return just the description in list format. Here's the text:\n\n "
#model_name = "gpt-3.5-turbo"
model_name="gpt-4"
temperature = 0
model = ChatOpenAI(temperature=temperature)


def code_data_with_langchain(df, model, parser, info_query_generic): 
  langchain_prompt = PromptTemplate(
    template="{query}\n",
    input_variables=["query"]
  )
  system_prompt = "You are a competent researcher who answers user queries."

  res = []
  #for index in range(2):
  for index, _ in df.iterrows():
    print(index)
    info_query = info_query_generic + df["text"][index] + "\n\n"
    _input = langchain_prompt.format_prompt(query=info_query)
    #print(_input)
    full_prompt=[SystemMessage(content=system_prompt), HumanMessage(content=_input.to_string())]
    #print(full_prompt)
    output = model(full_prompt)
    #parsed_output = parser.parse(output)
    string_output=output.content
    print(string_output)
    res.append(string_output)
  return res


res = code_data_with_langchain(input, model, parser,  info_query_generic)

output=input
output['gpt_out']=res
output.to_csv("data/post/rotate_6.csv")





0


['arms extended', 'one leg forward', 'one leg backwards', 'both legs on ground', 'right leg slightly higher', 'arms are head level', 'pointed slightly up', 'back leg is thicc', 'front leg is a triangle']
1
['one left straight on the floor', 'back leg pointed back at a 90 degree angle', 'arms forward', 'head tilted back', 'two triangle arms', 'one arm below the other']
2
['sat on the floor', 'knees up to the chest', 'diamond head', 'thicc body', 'like a triangle on a rectangle', 'they look pensive', 'like they might be waiting', 'you can clearly see legs']
3
['the most square kinda body', 'tiny little triangle arm', 'only one arm', 'only one foot', 'diamond head', 'ya boi be thicc', 'his left side is very rectangular', 'right side is where the right arm and leg is', 'strong penguin vibes']
4
['spooky ghost pose', 'both arms are together points to the left, like elbows high, hands down', 'one leg is flat on the floor', 'the other leg is raised to the right, in a straight line', 'his head

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Tue, 11 Jul 2023 16:27:19 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e52623329aacf21-SJC', 'alt-svc': 'h3=":443"; ma=86400'}.


['sad man, facing left sitting']
177
['ice skater, facing left']
178
['full kneeling facing right', 'triangular arms', 'arms out left and right']
179
['a guy is holding a cup of soup facing to the left of your screen']
180
['a person sitting against a wall facing to the RIGHT of your screen', 'two separate triangles on their head, kind of like wacky hair']
181
['a guy leaning backwards resting on his arms face the left of your screen', 'they have a rotated square and a triangle on their head', 'their "arm" is an upside down triangle', 'no like they are leaning backwards, half leaning half laying', 'there is an upside down triangle for their arm', 'they are facing the left of your screen', 'there is a rotated square on the left side of their head', 'a triangle rotated 45 degrees on the left side of their head']
182
['a guy in a sleeping bag with his head sticking out', 'their head is a rotated square, pointed']
183
['a guy doing a yoga pose facing to the left of the screen', 'standing',

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


...

['sitting indian', 'bottom completely flat']
590
['praying right', 'feet up behind']
591
['football kick', 'one foot on the ground', 'both feet to the left']
592
['classic ghost', 'both arms up', 'no legs', 'slanting back to the left']
593
['one foot on the ground', 'other foot kick to the right']
594
['triangle-square-triangle on top']
595
['left side all flat', 'diamond centered on top', 'feet and hands to the right']
596
['person sitting', 'facing left', 'wall on right', 'no arms', 'legs bent']
597
['lower left like hockey stick', 'arms like on a pole']
598
['a person half bending down with hands forward, facing down, on the floor, I would say to the right, it's in the centre of their body']
599
['quite chaotic', 'almost lying on floor with one arm towards floor and one in air', 'head facing to left']
600
['standing on one leg with other leg in the air to the right', '2 arms facing to the left pointing forward']
601
['person sat down with head facing to the left, legs facing to the 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


['On knees, hands to the right, straight posture']
736
['Weirdest figure', 'triangle ponytail']
737
['On legs', 'hands both sides', 'head more on the right', 'Like trying to fly or something but on knees']
738
['Goalkeeper', 'Hands up', 'trying to save the ball', 'Diving in air']
739
['Slightly bent legs', 'heading right', 'Flat image']
740
['Sitting like a monk', 'can\'t really see hands', 'heading right', 'Seem crossed', 'maybe can see knees', 'but it\'s safe to say no hands or legs']
741
['Sitting to left with legs slightly up, head looking at the legs']
742
['sitting facing left', 'knees bent', 'looking at legs']
743
['the weird triangle ponytail one']
744
['karate']
745
['facing left', 'small triangle for one leg', 'fat left trailing behind', 'another small triangle for hand', 'same as front foot', 'facing right']
746
['knees slightly bent', 'holding hands in front', 'facing right']
747
['penguin']
748
['bunny ears']
749
['no arms or legs']
750
['holding a box facing left']
751
['

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


['graceful pose', 'back bent', 'arm up and down']
943
['slender pose kneeling', 'to the right', 'tail up']
944
['body block', 'straight line on top of body']
945
['ice skater, looking left']
946
['sitting down', 'no arms or legs', 'looks like a sack']
947
['a mouse', 'head tilting to the right', 'triangle coming from it', 'the bunny ears', 'one arm to the left', 'the other triangle is attached to the bottom of the head', 'triangle arms look like wings almost', 'with head on top']
948
['dancing bear']
949
['body facing to the left', 'arm to the right', 'somebody is lying down leaning on one arm', 'head tilted to the left', 'no curve']
950
['kneeling down praying', 'tail upwards', 'arms to the right']
951
['standing up reading, arms to the left']
952
['body like a block', 'small triangle arm to the right', 'tiny tail to the right']
953
['bunny ears']
954
['priest is praying, but crouching, arms to the right']
955
['sitting down', 'curved back', 'sitting under a tree']
956
['goalkeeper']
